<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternating Least Squares

This notebook will ALS Matrix Factorization algorithm to recommend and rank Top10 items based on the paper of Koren, Bell and Volinsky, 2009 (https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf)

The used library:

*   Implicit ALS(https://benfred.github.io/implicit/ by Ben Frederickson)

In [14]:
!pip install --upgrade implicit

In [15]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta

# Loading DataFrames



In [16]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-16-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [17]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [18]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2220299 17697 77401


# Data Preprocessing

In [19]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)
# Entry Date to date
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]
# dropping backlog invoices, Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]
#insert purchase indication column
df["purchase"] = 1

In [20]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [21]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('BranchCustomerNbr')['Sku'].transform('nunique')

  return df_pl

In [22]:
df = sku_count(df)

In [23]:
# drop customers that only purchased 1 SKU
#df = df[df["sku_count"] > 1]

In [24]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [25]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [26]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,...,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,sku_count,bcn_id,sku_id
213,1545306,2022-11-02,NaN,2022FM11,15885514,AXIS SOLUTION (PRIVATE) LIMITED,Export Channel (DE),,CF55877,DT PRINT ZQ320 KIT LABEL SENSOR,...,5805,AIDC/PoS Printers,Mobile Receipt Printer,"Other (incl. AIDC/POS, V7)","10.713,30",41.0,1,20,497,49835
458,4422886,2022-11-03,NaN,2023FM02,44413224,BWG INFORMATIONSYSTEME GMBH,Business Channel,,CF89211,Z-SELECT 2000D REMOVABLE NS,...,5812,AIDC/PoS Printers,Label Printers Supplies,"Other (incl. AIDC/POS, V7)","1.393,00",140.0,1,111,1000,52759


In [27]:
users = df.bcn_id.unique()
skus = df.sku_id.unique()
print(len(users), len(skus))

13894 75643


In [28]:
df_2 = df[["bcn_id", "sku_id", "Entry Date", "Qty Shipped", "purchase"]]

In [29]:
grouped_df = df_2.groupby(["bcn_id", "sku_id"]).agg({
          "Qty Shipped":"sum",
          "purchase":"sum"}).reset_index()

In [30]:
# create binary column
grouped_df["purch_bin"] = 1

In [31]:
grouped_df.head(2)

,bcn_id,sku_id,Qty Shipped,purchase,purch_bin
0,0,928,1.0,1,1
1,0,3422,1.0,1,1


In [32]:
grouped_df_2 = grouped_df.copy()

## Negative Sampling

In [33]:
def negative_sampling(df_pl, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_pl = pd.concat([df_pl, df_neg], ignore_index=True)

    return df_pl

In [34]:
neg_grouped_df = negative_sampling(grouped_df_2, users, skus, skus, n_neg=100)

In [35]:
neg_grouped_df = neg_grouped_df.fillna(0)

In [36]:
print(len(grouped_df), len(neg_grouped_df))

544417 1933817


## ALS Model param tuning

In [37]:
import implicit
from implicit.gpu.als import AlternatingLeastSquares as ALS
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split

In [38]:
csr_qty_matrix = sparse.csr_matrix((grouped_df['Qty Shipped'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_freq_matrix = sparse.csr_matrix((grouped_df['purchase'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_bin_matrix = sparse.csr_matrix((grouped_df['purch_bin'], (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [39]:
neg_csr_qty_matrix = sparse.csr_matrix((neg_grouped_df['Qty Shipped'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_freq_matrix = sparse.csr_matrix((neg_grouped_df['purchase'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_bin_matrix = sparse.csr_matrix((neg_grouped_df['purch_bin'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))

In [40]:
csr_qty_matrix_train, csr_qty_matrix_test = leave_k_out_split(csr_qty_matrix, K=1, random_state=None)
csr_freq_matrix_train, csr_freq_matrix_test = leave_k_out_split(csr_freq_matrix, K=1, random_state=None)
csr_bin_matrix_train, csr_bin_matrix_test = leave_k_out_split(csr_bin_matrix, K=1, random_state=None)

In [41]:
csr_qty_matrix_train_2, csr_qty_matrix_test_2 = train_test_split(csr_qty_matrix, train_percentage=0.9, random_state=None)
csr_freq_matrix_train_2, csr_freq_matrix_test_2 = train_test_split(csr_freq_matrix, train_percentage=0.9,  random_state=None)
csr_bin_matrix_train_2, csr_bin_matrix_test_2 = train_test_split(csr_bin_matrix, train_percentage=0.9, random_state=None)

In [42]:
neg_csr_qty_matrix_train, neg_csr_qty_matrix_test = leave_k_out_split(neg_csr_qty_matrix, K=1, random_state=None)
neg_csr_freq_matrix_train, neg_csr_freq_matrix_test = leave_k_out_split(neg_csr_freq_matrix, K=1, random_state=None)
neg_csr_bin_matrix_train, neg_csr_bin_matrix_test = leave_k_out_split(neg_csr_bin_matrix, K=1, random_state=None)

In [43]:
model = ALS()

In [44]:
model.fit(csr_freq_matrix_train_2)

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
auc10 = AUC_at_k(model, csr_freq_matrix_train_2, csr_freq_matrix_test_2, K=10, num_threads=10)
auc10

  0%|          | 0/6924 [00:00<?, ?it/s]

0.5529487148551209

## Hyperparametertuning


In [119]:
from IPython.utils.sysinfo import num_cpus
from sklearn.model_selection import ParameterGrid

# Grid of hyperparameters to search
param_grid = {
    'factors': [100, 150, 200],
    'iterations':[10, 15, 20],
    'alpha': [5, 10, 20, 40],
    'regularization': [0.01, 0.1]
}

best_auc = -np.inf
best_params = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model.fit(csr_qty_matrix_train, show_progress=False)

    auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10 > best_auc:
        best_auc = auc10
        best_params = params

# Print the best parameters and AUC
print("Best parameters:", best_params)
print("Best AUC:", best_auc)

Best parameters: {'alpha': 20, 'factors': 200, 'iterations': 20, 'regularization': 0.01}
Best AUC: 0.5866738109246707


In [121]:
best_auc_2 = -np.inf
best_params_2 = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model_2 = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model_2.fit(csr_freq_matrix_train, show_progress=False)

    auc10_2 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10_2 > best_auc_2:
        best_auc_2 = auc10_2
        best_params_2 = params

# Print the best parameters and AUC
print("Best parameters:", best_params_2)
print("Best AUC:", best_auc_2)

Best parameters: {'alpha': 10, 'factors': 200, 'iterations': 10, 'regularization': 0.01}
Best AUC: 0.5932505860850016


In [122]:
model = ALS(**best_params)
model.fit(csr_qty_matrix_train)
auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.5863113789595255; Prec@10: 0.17274233637116818; Map@10: 0.0958602102812958; Ndcg@10: 0.11410104929923907


In [124]:
model_2 = ALS(**best_params_2)
model_2.fit(csr_freq_matrix_train)
auc10 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.5928881486433984; Prec@10: 0.18589478044739022; Map@10: 0.10695529385463125; Ndcg@10: 0.12558158832741184


## Recommending with ALS

In [53]:
# add column that represent sku count
def sku_count_2(df_pl):

  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')

  return df_pl

In [55]:
df_2 = sku_count_2(df_2)

<ipython-input-53-773a5c2f08df>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')


In [56]:
users_less_20 = df_2[(df_2.sku_count <= 20) & (df_2.sku_count >= 5) ]["bcn_id"].unique()
users_more_20 = df_2[df_2.sku_count > 20]["bcn_id"].unique()

In [60]:
sku_list = df[["sku_id", "Product Descr1", "ProductGroupDescription", "ProductGroupMasterDescription"]].drop_duplicates()
sku_list.head(1)

,sku_id,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
213,49835,DT PRINT ZQ320 KIT LABEL SENSOR,Mobile Receipt Printer,AIDC/PoS Printers


In [103]:
from google.colab import files

In [97]:
 # Assuming you want recommendations for user with ID 10
import random
user_id = random.choice(users_less_20)
user_id_2 = random.choice(users_more_20)
print(user_id)
print(user_id_2)

3417
1532


In [105]:
user_purchases = pd.DataFrame(df_2[(df.bcn_id == user_id)])
user_purchases = user_purchases.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases = user_purchases.sort_values(by="Qty Shipped", ascending=False)
user_purchases = user_purchases.merge(sku_list, on="sku_id", how="left")
user_purchases.to_csv(f'{user_id}_purchase_history.csv')
files.download(f'{user_id}_purchase_history.csv')
user_purchases

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,bcn_id,sku_id,Qty Shipped,purchase,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,3417,3451,1.0,1,2TB RED 256MB SMR,Hard Drive Sata,Hard Drives & Optical Drives
1,3417,6314,1.0,1,BACK-UPS PRO 1500 POWER-SAVING,Uninterruptable Power Supplies,Power Equipment (UPS)
2,3417,28590,1.0,1,TFT U2722D 27IN IPS,Lcd Monitor,Display
3,3417,46438,1.0,1,OPTIPLEX 3000 MT TPM I5-12500,Desktop/Tower Computers,Computer Systems
4,3417,49961,1.0,1,ESPRIMO G6012 HS I5-12400T 8GB,Desktop/Tower Computers,Computer Systems


In [115]:
user_purchases_2 = pd.DataFrame(df_2[(df.bcn_id == user_id_2)])
user_purchases_2 = user_purchases_2.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases_2 = user_purchases_2.sort_values(by="Qty Shipped", ascending=False)
user_purchases_2 = user_purchases_2.merge(sku_list, on="sku_id", how="left")
user_purchases_2.to_csv(f'{user_id_2}_purchase_history.csv')
files.download(f'{user_id_2}_purchase_history.csv')
len(user_purchases_2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

229

## Recommendations

#### Model 1

In [130]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id}_ALS_QTY_REC.csv')
files.download(f'{user_id}_ALS_QTY_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,46416.0,0.013647,LATITUDE 5530 I5-1245U,Portable/Notebook Computers,Computer Systems
1,2428.0,0.012546,TFT P2421 24IN LCD BLACK,Lcd Monitor,Display
2,33754.0,0.011991,TFT P2722H 27IN IPS BK,Lcd Monitor,Display
3,46433.0,0.011745,OPTIPLEX 3000 MFF TPM I5-12500T,Desktop/Tower Computers,Computer Systems
4,46431.0,0.011490,OPTIPLEX 3000 MFF TPM I5-12500T,Desktop/Tower Computers,Computer Systems
5,30452.0,0.011376,TFT P2722HE 27IN IPS BK,Lcd Monitor,Display
6,28593.0,0.011101,TFT U2722DE 27IN IPS,Lcd Monitor,Display
7,48436.0,0.011066,SCREEN PROTECTOR ANTI-SHOCK,Screen Protectors,Mobility
8,28585.0,0.010821,TFT U2422H 23.8IN IPS,Lcd Monitor,Display
9,51582.0,0.010676,TFT U2722D 27IN IPS,Lcd Monitor,Display


In [131]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id_2}_ALS_QTY_REC.csv')
files.download(f'{user_id_2}_ALS_QTY_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,75531.0,1.201274,BU FOR OFFICE365 COM,Back-Up Software,Softwares
1,8804.0,0.930354,ARUBA AP-505 (RW) UNIFIED STOCK,Wireless Lan Access Points (Ap),Communications & Networking
2,8582.0,0.843682,AP-MNT-E AP MOUNT BRACKET STOCK,Mounting Hardware,Accessories
3,8014.0,0.803815,ARUBA 10G SFP+3M CABL-STOCK,Network Ethernet Cables & Adapters,Cables
4,8013.0,0.789966,10G SFP+ TO SFP+ 1M DAC-STOCK,Network Fiber Optic Cables & Adapters,Cables
5,7540.0,0.785157,ARUBA 6100 48G CL4 4SFP+ STOCK,Lan Switches Managed Layer 3,Communications & Networking
6,8955.0,0.757628,PC-AC-EC CONT EURO AC-STOCK,Power Cables,Cables
7,8472.0,0.679533,ARUBA AP-535 (RW) UNIFIED STOCK,Wireless Lan Access Points (Ap),Communications & Networking
8,22480.0,0.672686,X130 10G SFP+ LC SR TRX-STOCK,"Transceivers (Gbic, Spf) & Media Converters",Communications & Networking
9,7404.0,0.665513,2930 2-PORT STACKING MOD STOCK,Lan Switches Managed Layer 2,Communications & Networking


#### Model 2

In [134]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model_2.recommend(userid, csr_freq_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id}_FREQ_ALS_REC.csv')
files.download(f'{user_id}_FREQ_ALS_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,35225.0,0.026717,OFFICE HOME AND BUSINESS 2021,Desktop Operating System Software,Softwares
1,33752.0,0.022314,TFT P2422H 24IN IPS BK,Lcd Monitor,Display
2,33754.0,0.020552,TFT P2722H 27IN IPS BK,Lcd Monitor,Display
3,30442.0,0.019587,TFT P2422H 24IN IPS BK,Lcd Monitor,Display
4,46428.0,0.019075,OPTIPLEX 3000 MFF TPM I3-12100T,Desktop/Tower Computers,Computer Systems
5,46431.0,0.018464,OPTIPLEX 3000 MFF TPM I5-12500T,Desktop/Tower Computers,Computer Systems
6,46433.0,0.018461,OPTIPLEX 3000 MFF TPM I5-12500T,Desktop/Tower Computers,Computer Systems
7,46416.0,0.018064,LATITUDE 5530 I5-1245U,Portable/Notebook Computers,Computer Systems
8,2428.0,0.017910,TFT P2421 24IN LCD BLACK,Lcd Monitor,Display
9,46436.0,0.017221,OPTIPLEX 3000 SFF TPM I5-12500,Desktop/Tower Computers,Computer Systems


In [136]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model_2.recommend(userid, csr_freq_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id_2}_FREQ_ALS_REC.csv')
files.download(f'{user_id_2}_FREQ_ALS_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,sku_id,score,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
0,8804.0,0.770070,ARUBA AP-505 (RW) UNIFIED STOCK,Wireless Lan Access Points (Ap),Communications & Networking
1,7540.0,0.719754,ARUBA 6100 48G CL4 4SFP+ STOCK,Lan Switches Managed Layer 3,Communications & Networking
2,8014.0,0.657472,ARUBA 10G SFP+3M CABL-STOCK,Network Ethernet Cables & Adapters,Cables
3,8013.0,0.629655,10G SFP+ TO SFP+ 1M DAC-STOCK,Network Fiber Optic Cables & Adapters,Cables
4,75531.0,0.593446,BU FOR OFFICE365 COM,Back-Up Software,Softwares
5,9129.0,0.578658,HPE SOFTWARE UPDATES SVC,Services And Warranties,Softwares
6,8582.0,0.576901,AP-MNT-E AP MOUNT BRACKET STOCK,Mounting Hardware,Accessories
7,9128.0,0.574163,HPE TELEFONISCHE SW SUPPORT,Services And Warranties,Computer Systems
8,58892.0,0.546362,FACTORY INTEGRATED - CTO,Services And Warranties,Computer Systems
9,8006.0,0.531023,1G SFP LC SX 500M MMF-STOCK,"Transceivers (Gbic, Spf) & Media Converters",Communications & Networking
